# Micro-Net re-implementation

### Installations

In [ ]:
!pip install tensorflow
!pip install tensorpack
!pip install tensorflow-gpu

     |████████████████████████████████| 462 kB 4.2 MB/s 
     |████████████████████████████████| 296 kB 4.2 MB/s 
     |████████████████████████████████| 497.5 MB 26 kB/s 


In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


### Imports

In [ ]:
import importlib
import random
import cv2
import tensorflow as tf
from tensorpack import imgaug
import os
import scipy.io as sio
import numpy as np
from google.colab import drive
import math
import os
from collections import deque
import cv2
import numpy as np
from tensorpack.predict import OfflinePredictor, PredictConfig
from tensorpack.tfutils.sessinit import get_model_loader
import json
import operator
from scipy.ndimage import filters, measurements
from scipy.ndimage.morphology import (
                                    binary_erosion,
                                    binary_dilation, 
                                    binary_fill_holes,
                                    distance_transform_cdt,
                                    distance_transform_edt)
from skimage.morphology import remove_small_objects, watershed
import importlib
import random
import cv2
import numpy as np
import tensorflow as tf
from tensorpack import imgaug



### Load data & folder paths
Press on the folder symbol to the right of the notebook and then look for "mount drive". The folders used below should be in the root folder.

First place the dataset folder from the github repo into your drive (change "../dataset" on each row to "/content/drive/MyDrive/dataset" in each of the .txt files in the dataset folder, easy using replace in vs code)

then place the papillary RCC subtyping dataset(1,2, nuclei_prediction) in the same folder

Download the two model files and place them in the root folder

link: https://nextcloud.chenli.group/index.php/s/c4HzzCWXep5B24S
passwd: KrfmCJen

The dataset can be found using the link
https://dataset.chenli.group/home/prcc-subtyping
where you should download the dataset folder. Take its content of folder "1" and "2" and place in the already created datafolder

The structure will look like this

* drive (folder)
 * MyDrive (folder)
  * dataset (folder)
   * 1 (folder with images )
   * 2 (folder with images)
   * nuclei_prediction (folder)
   dataset.txt
   testset.txt
   trainset.txt
   validset.txt
  
   model-36000.data-00000-of-00001
   
   model-36000.index


out_dir changed to the folder where we put the nuclei_prediction folder

In [ ]:
# Connect to a drive which have the folders imported 
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/dataset/dataset.txt'
out_dir = '/content/drive/MyDrive/dataset/nuclei_prediction/'

### utils.py

In [ ]:

import math
import numpy as np

import tensorflow as tf

from tensorpack import *
from tensorpack.tfutils.symbolic_functions import *
from tensorpack.tfutils.summary import *

from matplotlib import cm

# TODO: assert for data format
####
def resize_op(x, height_factor=None, width_factor=None, size=None, 
                interp='bicubic', data_format='channels_last'):
    """
    Resize by a factor if `size=None` else resize to `size`
    """
    original_shape = x.get_shape().as_list()
    if size is not None:
        if data_format == 'channels_first':
            x = tf.transpose(x, [0, 2, 3, 1])
            if interp == 'bicubic':
                x = tf.image.resize_bicubic(x, size)
            elif interp == 'bilinear':
                x = tf.image.resize_bilinear(x, size)
            else:
                x = tf.image.resize_nearest_neighbor(x, size)
            x = tf.transpose(x, [0, 3, 1, 2])
            x.set_shape((None, 
                original_shape[1] if original_shape[1] is not None else None, 
                size[0], size[1]))
        else:
            if interp == 'bicubic':
                x = tf.image.resize_bicubic(x, size)
            elif interp == 'bilinear':
                x = tf.image.resize_bilinear(x, size)
            else:
                x = tf.image.resize_nearest_neighbor(x, size)
            x.set_shape((None, 
                size[0], size[1], 
                original_shape[3] if original_shape[3] is not None else None))
    else:
        if data_format == 'channels_first':
            new_shape = tf.cast(tf.shape(x)[2:], tf.float32)    
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('float32'))
            new_shape = tf.cast(new_shape, tf.int32)    
            x = tf.transpose(x, [0, 2, 3, 1])
            if interp == 'bicubic':
                x = tf.image.resize_bicubic(x, new_shape)
            elif interp == 'bilinear':
                x = tf.image.resize_bilinear(x, new_shape)
            else:
                x = tf.image.resize_nearest_neighbor(x, new_shape)
            x = tf.transpose(x, [0, 3, 1, 2])
            x.set_shape((None,
                        original_shape[1] if original_shape[1] is not None else None,
                        int(original_shape[2] * height_factor) if original_shape[2] is not None else None,
                        int(original_shape[3] * width_factor) if original_shape[3] is not None else None))
        else:
            original_shape = x.get_shape().as_list()
            new_shape = tf.cast(tf.shape(x)[1:3], tf.float32)    
            new_shape *= tf.constant(np.array([height_factor, width_factor]).astype('float32'))
            new_shape = tf.cast(new_shape, tf.int32)    
            if interp == 'bicubic':
                x = tf.image.resize_bicubic(x, new_shape)
            elif interp == 'bilinear':
                x = tf.image.resize_bilinear(x, new_shape)
            else:
                x = tf.image.resize_nearest_neighbor(x, new_shape)
            x.set_shape((None,
                        int(original_shape[1] * height_factor) if original_shape[1] is not None else None,
                        int(original_shape[2] * width_factor) if original_shape[2] is not None else None,
                        original_shape[3] if original_shape[3] is not None else None))
    return x 

####
def crop_op(x, cropping, data_format='channels_first'):
    """
    Center crop image
    Args:
        cropping is the substracted portion
    """
    crop_t = cropping[0] // 2
    crop_b = cropping[0] - crop_t
    crop_l = cropping[1] // 2
    crop_r = cropping[1] - crop_l
    if data_format == 'channels_first':
        x = x[:,:,crop_t:-crop_b,crop_l:-crop_r]
    else:
        x = x[:,crop_t:-crop_b,crop_l:-crop_r]
    return x       
####

def categorical_crossentropy(output, target):
    """
        categorical cross-entropy, accept probabilities not logit
    """
    # scale preds so that the class probs of each sample sum to 1
    output /= tf.reduce_sum(output,
                            reduction_indices=len(output.get_shape()) - 1,
                            keepdims=True)
    # manual computation of crossentropy
    epsilon = tf.convert_to_tensor(10e-8, output.dtype.base_dtype)
    output = tf.clip_by_value(output, epsilon, 1. - epsilon)
    return - tf.reduce_sum(target * tf.log(output),
                            reduction_indices=len(output.get_shape()) - 1)
####
def dice_loss(output, target, loss_type='sorensen', axis=None, smooth=1e-3):
    """Soft dice (Sørensen or Jaccard) coefficient for comparing the similarity
    of two batch of data, usually be used for binary image segmentation
    i.e. labels are binary. The coefficient between 0 to 1, 1 means totally match.
    Parameters
    -----------
    output : Tensor
        A distribution with shape: [batch_size, ....], (any dimensions).
    target : Tensor
        The target distribution, format the same with `output`.
    loss_type : str
        ``jaccard`` or ``sorensen``, default is ``jaccard``.
    axis : tuple of int
        All dimensions are reduced, default ``[1,2,3]``.
    smooth : float
        This small value will be added to the numerator and denominator.
            - If both output and target are empty, it makes sure dice is 1.
            - If either output or target are empty (all pixels are background), 
              dice = ```smooth/(small_value + smooth)``, then if smooth is very small, 
              dice close to 0 (even the image values lower than the threshold), 
              so in this case, higher smooth can have a higher dice.
    Examples
    ---------
    >>> dice_loss = dice_coe(outputs, y_)
    """
    target = tf.squeeze(tf.cast(target, tf.float32))
    output = tf.squeeze(tf.cast(output, tf.float32))

    inse = tf.reduce_sum(output * target, axis=axis)
    if loss_type == 'jaccard':
        l = tf.reduce_sum(output * output, axis=axis)
        r = tf.reduce_sum(target * target, axis=axis)
    elif loss_type == 'sorensen':
        l = tf.reduce_sum(output, axis=axis)
        r = tf.reduce_sum(target, axis=axis)
    else:
        raise Exception("Unknown loss_type")
    # already flatten
    dice = 1.0 - (2. * inse + smooth) / (l + r + smooth)
    ##
    return dice
####
def colorize(value, vmin=None, vmax=None, cmap=None):
    """
    Arguments:
      - value: input tensor, NHWC ('channels_last')
      - vmin: the minimum value of the range used for normalization.
        (Default: value minimum)
      - vmax: the maximum value of the range used for normalization.
        (Default: value maximum)
      - cmap: a valid cmap named for use with matplotlib's `get_cmap`.
        (Default: 'gray')
    Example usage:
    ```
    output = tf.random_uniform(shape=[256, 256, 1])
    output_color = colorize(output, vmin=0.0, vmax=1.0, cmap='viridis')
    tf.summary.image('output', output_color)
    ```
    
    Returns a 3D tensor of shape [height, width, 3], uint8.
    """

    # normalize
    if vmin is None:
        vmin = tf.reduce_min(value, axis=[1,2])
        vmin = tf.reshape(vmin, [-1, 1, 1])
    if vmax is None:
        vmax = tf.reduce_max(value, axis=[1,2])
        vmax = tf.reshape(vmax, [-1, 1, 1])
    value = (value - vmin) / (vmax - vmin) # vmin..vmax

    # squeeze last dim if it exists
    # NOTE: will throw error if use get_shape()
    # value = tf.squeeze(value)

    # quantize
    value = tf.round(value * 255)
    indices = tf.cast(value, np.int32)

    # gather
    colormap = cm.get_cmap(cmap if cmap is not None else 'gray')
    colors = colormap(np.arange(256))[:, :3]
    colors = tf.constant(colors, dtype=tf.float32)
    value = tf.gather(colors, indices)
    value = tf.cast(value * 255, tf.uint8)
    return value
####
def make_image(x, cy, cx, scale_y, scale_x):
    """
    Take 1st image from x and turn channels representations
    into 2D image, with cx number of channels in x-axis and
    cy number of channels in y-axis
    """
    # norm x for better visual
    x = tf.transpose(x,(0,2,3,1)) # NHWC
    max_x = tf.reduce_max(x, axis=-1, keep_dims=True)
    min_x = tf.reduce_min(x, axis=-1, keep_dims=True)
    x = 255 * (x - min_x) / (max_x - min_x)
    ###
    x_shape = tf.shape(x)
    channels = x_shape[-1]
    iy , ix = x_shape[1], x_shape[2] 
    ###
    x = tf.slice(x,(0,0,0,0),(1,-1,-1,-1))
    x = tf.reshape(x,(iy,ix,channels))
    ix += 4
    iy += 4
    x = tf.image.resize_image_with_crop_or_pad(x, iy, ix)
    x = tf.reshape(x,(iy,ix,cy,cx)) 
    x = tf.transpose(x,(2,0,3,1)) #cy,iy,cx,ix
    x = tf.reshape(x,(1,cy*iy,cx*ix,1))
    x = resize_op(x, scale_y, scale_x)
    return tf.cast(x, tf.uint8)
####

### Config.py
Contains:
* Config class
 * Dependent on utils.py

changed

        config_file = importlib.import_module('other') # fcn8, dcan, etc.
        config_dict = config_file.__getattribute__(self.model_type)

to

        config_dict = {
    'train_input_shape' : [252, 252],
    'train_mask_shape'  : [252, 252],
    'infer_input_shape' : [252, 252],
    'infer_mask_shape'  : [252, 252], 
    'infer_avalible_shape': [200, 200],

    'training_phase'    : [
        {
            'nr_epochs': 80,
            'manual_parameters' : {
                'learning_rate': (1.0e-4, [('40', 1.0e-5)]),
                'aux_loss_dw'  : (1.0, 
                        [(str(epoch), 1.0 / epoch) for epoch in range(2, 251)]
                    ),
            },
            'train_batch_size'  : 8,
            'infer_batch_size'  : 16,

            'model_flags' : {
            }
        }
    ],

    'optimizer'         : tf.compat.v1.train.Optimizer,

    'inf_batch_size'  : 16,
    'inf_auto_metric'   : 'valid_dice',
    'inf_auto_comparator' : '>',}


This works because the model_type is hardcoded to "micronet". The dict can be found in other.py with the tensorflow version modification of 

      tf.train.AdamOptimizer 
to

    tf.compat.v1.train.Optimizer,


We also had to change

      model_constructor = importlib.import_module('%s' % self.model_type)
      model_constructor = model_constructor.Graph       
      return model_constructor # NOTE return alias, not object

where we return the Graph class in the micronet.py directly in the code. 

and lastly

      self.save_dir = './%s' % self.model_type

to

      self.save_dir = './%s' % 'content/drive/MyDrive'



The below changes are mentioned in https://github.com/tensorpack/tensorpack/blob/master/CHANGES.md

* In the Graph class i changed the name of _get_inputs() to input() because it threw a not implemented error for the interface ModelDesc

the same goes for 
* InputDesc which should be replaced with tf.TensorSpec(Also switch first and second argument) 

* _get_optimizer() should be named optimizer()

* _build_graph() should be named build_graph()


Another change we had to do was to modify the build_graph signature from

      def build_graph(self, input):
        ...
        images, truemap_coded = inputs

to

      def build_graph(self, images, truemap_coded):

because it was complaining about to few arguments. The reason for this is probably the changed return value from inputs()
                
Since we are now using tensorflow 2.x we also had to change

      tf.contrib
to

      tf.compat.v1.estimator




In [ ]:
class Config(object):
    def __init__(self, ):

        self.seed = 10 
        mode = 'micronet'
        self.model_type = 'micronet'

        
        self.type_boundray = True # whether to predict the nuclear boundray
        self.type_nuclei = True
        self.type_classification = True # whether to predict the nuclear type
        # ! must use CoNSeP dataset, where nuclear type labels are available
        # denotes number of classes for nuclear type classification, 
        # plus the background class
        self.nr_types = 5
        # ! some semantic segmentation network like micronet,
        # ! nr_types will replace nr_classes if type_classification=True
        self.nr_classes = 2 # Nuclei Pixels vs Background
        
        self.auxilary_tasks = False # whether to use deep supervision or auxilary tasks strategy
        self.regression = True # whether to use np-regression branch
        self.gcb = True # whether to use attention block on encoder
        self.uncertainty = False # whether to use uncertainty loss weight strategy
        self.mix_class = True # single or multiple classification branches
        self.use_dice = False # whether to use dice loss

        # define your nuclei type name here, please ensure it contains
        # same the amount as defined in `self.nr_types` . ID 0 is preserved
        # for background so please don't use it as ID
        self.nuclei_type_dict = {
            'Grade1': 1, # ! Please ensure the matching ID is unique
            'Grade2': 2,
            'Grade3' : 3,
            'Endothelium': 4,
        }
        assert len(self.nuclei_type_dict.values()) == self.nr_types - 1

        #### Dynamically setting the config file into variable
        config_dict = {
    'train_input_shape' : [252, 252],
    'train_mask_shape'  : [252, 252],
    'infer_input_shape' : [252, 252],
    'infer_mask_shape'  : [252, 252], 
    'infer_avalible_shape': [200, 200],

    'training_phase'    : [
        {
            'nr_epochs': 80,
            'manual_parameters' : {
                'learning_rate': (1.0e-4, [('40', 1.0e-5)]),
                'aux_loss_dw'  : (1.0, 
                        [(str(epoch), 1.0 / epoch) for epoch in range(2, 251)]
                    ),
            },
            'train_batch_size'  : 8,
            'infer_batch_size'  : 16,

            'model_flags' : {
            }
        }
    ],

    'optimizer'         : tf.compat.v1.train.Optimizer,

    'inf_batch_size'  : 16,
    'inf_auto_metric'   : 'valid_dice',
    'inf_auto_comparator' : '>',
    }

        for variable, value in config_dict.items():
            self.__setattr__(variable, value)
        #### Training data

        self.data_ext = '.npy' 

        # number of processes for parallel processing input
        self.nr_procs_train = 8 
        self.nr_procs_valid = 4 

        self.input_norm  = True # normalize RGB to 0-1 range

        ####
        self.save_dir = '/content/drive/MyDrive'

        # path to checkpoints will be used for inference, replace accordingly
        self.inf_model_path  = self.save_dir + '/model-36000.index'



        # output will have channel ordering as [Nuclei Type][Nuclei Pixels][Additional]
        # where [Nuclei Type] will be used for getting the type of each instance
        # while [Nuclei Pixels][Additional] will be used for extracting instances
        # for inference during evalutaion mode i.e run by infer.py
        self.eval_inf_input_tensor_names = ['images']
        self.eval_inf_output_tensor_names = ['predmap-coded']
        # for inference during training mode i.e run by trainer.py
        self.train_inf_output_tensor_names = ['predmap-coded', 'truemap-coded']

        self.inf_imgs_ext = '.png'

    def get_model(self):
        class Graph(ModelDesc, Config):
            def __init__(self):
                super(Graph, self).__init__()
                assert tf.test.is_gpu_available()
                self.data_format = 'channels_first'

            def inputs(self):
                return [tf.TensorSpec([None] + self.train_input_shape + [3], tf.float32, 'images'),
                        tf.TensorSpec([None] + self.train_mask_shape  + [None], tf.float32,  'truemap-coded')]

            # for node to receive manual info such as learning rate.
            def add_manual_variable(self, name, init_value, summary=True):
                var = tf.get_variable(name, initializer=init_value, trainable=False)
                if summary:
                    tf.summary.scalar(name + '-summary', var)
                return

            def get_optimizer(self):
                with tf.variable_scope("", reuse=True):
                    lr = tf.get_variable('learning_rate')
                opt = self.optimizer(learning_rate=lr)
                return opt

            def build_graph(self, images, truemap_coded):

                ####
                is_training = get_current_tower_context().is_training
                
               # images, truemap_coded = inputs

                orig_imgs = images
                pen_map = truemap_coded[...,-1]
                if self.type_classification:
                    true = truemap_coded[...,1]
                else:
                    true = truemap_coded[...,0]            
                true = tf.cast(true, tf.int32)
                true = tf.identity(true, name='truemap')
                one  = tf.one_hot(true, self.nr_types if self.type_classification else self.nr_classes, axis=-1)
                true = tf.expand_dims(true, axis=-1)

                def down_branch(name, main_in, aux_in, ch):
                    with tf.variable_scope(name):
                        a = Conv2D('conv1', main_in, ch, 3, padding='valid', use_bias=False, activation=BNReLU)
                        a = Conv2D('conv2', a, ch, 3, padding='valid', use_bias=True, activation=tf.nn.relu)
                        a = MaxPooling('pool', a, 2, strides=2, padding= 'same') 

                        b = Conv2D('conv3', aux_in, ch, 3, padding='valid', use_bias=False, activation=BNReLU)
                        b = Conv2D('conv4',      b, ch, 3, padding='valid', use_bias=True, activation=tf.nn.relu)

                        c = tf.concat([a, b], axis=1)
                    return c

                def up_branch(name, main_in, aux_in, ch):
                    with tf.variable_scope(name):
                        a = Conv2DTranspose('up1', main_in, ch, 2, strides=(2, 2), padding='same', use_bias=True, activation=tf.identity)
                        a = Conv2D('conv1', a, ch, 3, padding='valid', use_bias=True, activation=tf.nn.relu)
                        a = Conv2D('conv2', a, ch, 3, padding='valid', use_bias=True, activation=tf.nn.relu)

                        # stride 1 is no different from normal 5x5 conv, 'valid' to gain extrapolated border pixels
                        b1 = Conv2DTranspose('up2',      a, ch, 5, strides=(1, 1), padding='valid', use_bias=True, activation=tf.identity)
                        b2 = Conv2DTranspose('up3', aux_in, ch, 5, strides=(1, 1), padding='valid', use_bias=True, activation=tf.identity)
                        b = tf.concat([b1, b2], axis=1)
                        b = Conv2D('conv3', b, ch, 1, padding='same', use_bias=True, activation=tf.nn.relu)
                    return b

                def aux_branch(name, main_in, up_kernel, up_strides):
                    ch = main_in.get_shape().as_list()[1] # NCHW
                    with tf.variable_scope(name): # preserve the depth
                        a = Conv2DTranspose('up', main_in, ch, up_kernel, strides=up_strides, padding='same', use_bias=True, activation=tf.identity)
                        a = Conv2D('conv', a, self.nr_types if self.type_classification else self.nr_classes, 3, padding='valid', activation=tf.nn.relu)
                        a = tf.layers.dropout(a, rate=0.5, seed=5, training=is_training)
                    return a

                #### Xavier initializer
                with argscope(Conv2D, activation=tf.identity, 
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(uniform=True),
                            bias_initializer=tf.constant_initializer(0.1)), \
                    argscope(Conv2DTranspose, activation=tf.identity, 
                            kernel_initializer=tf.contrib.layers.xavier_initializer_conv2d(uniform=True),
                            bias_initializer=tf.constant_initializer(0.1)), \
                        argscope([Conv2D, Conv2DTranspose, MaxPooling, BatchNorm], data_format=self.data_format):

                    i = tf.transpose(images / 255.0, [0, 3, 1, 2]) # our way
                    resize_func = lambda x, y: resize_op(x, size=y,interp='bicubic', data_format='channels_first')

                    ####
                    b1 = down_branch('b1',  i, resize_func(i, (128, 128)),  64)
                    b2 = down_branch('b2', b1, resize_func(i, ( 64,  64)), 128)
                    b3 = down_branch('b3', b2, resize_func(i, ( 32,  32)), 256)
                    b4 = down_branch('b4', b3, resize_func(i, ( 16,  16)), 512)

                    with tf.variable_scope('b5'):
                        b5 = Conv2D('conv1', b4, 2048, 3, padding='valid', use_bias=True, activation=tf.nn.relu)
                        b5 = Conv2D('conv2', b5, 2048, 3, padding='valid', use_bias=True, activation=tf.nn.relu)
                    b6 = up_branch('b6', b5, b4, 1024)
                    b7 = up_branch('b7', b6, b3, 512)
                    b8 = up_branch('b8', b7, b2, 256)
                    b9 = up_branch('b9', b8, b1, 128)

                    aux_out1 = aux_branch('aux_out1', b9, 2, (2, 2))
                    aux_out2 = aux_branch('aux_out2', b8, 4, (4, 4))
                    aux_out3 = aux_branch('aux_out3', b7, 8, (8, 8))
                    out = tf.concat([aux_out1, aux_out2, aux_out3], axis=1)
                    out_list = [out, aux_out1, aux_out2, aux_out3]

                    soft_list = []
                    prob_list = []
                    for idx, sub_out in enumerate(out_list):
                        logi = Conv2D('conv_out%d' % idx, sub_out, 
                                        self.nr_types if self.type_classification else self.nr_classes, 
                                        3, padding='valid', use_bias=True, activation=tf.identity)
                        logi = tf.transpose(logi, [0, 2, 3, 1])
                        soft = tf.nn.softmax(logi, axis=-1)

                        if self.type_classification:
                            prob_np = tf.reduce_sum(soft[...,1:], axis=-1, keepdims=True)
                            prob_np = tf.identity(prob_np, name='predmap-prob-np')
                        else:
                            prob_np = tf.identity(soft[...,1], name='predmap-prob')
                            prob_np = tf.expand_dims(prob_np, axis=-1)
                        
                        soft_list.append(soft)
                        prob_list.append(prob_np)

                    # return the aggregated output
                    # encoded so that inference can extract all output at once
                    if self.type_classification:
                        predmap_coded = tf.concat([soft_list[0], prob_list[0]], axis=-1, name='predmap-coded')
                    else:
                        predmap_coded = tf.identity(prob_list[0], name='predmap-coded')

                ####
                if is_training:
                    ######## LOSS                       
                    # get the variable to received fed weight from external scheduler
                    with tf.variable_scope("", reuse=True):
                        aux_loss_dw = tf.get_variable('aux_loss_dw')

                    loss_list = [] # index 0 is main output
                    global_step = tf.train.get_or_create_global_step()
                    global_step = tf.cast(global_step, tf.float32)
                    for idx, sub_soft in enumerate(soft_list):
                        loss_bce = categorical_crossentropy(sub_soft, one)
                        loss_bce = tf.reduce_mean(loss_bce * pen_map)
                        loss_bce = loss_bce if idx == 0 else loss_bce * aux_loss_dw
                        loss_bce = tf.identity(loss_bce, name='loss-bce-%d' % idx)
                        loss_list.append(loss_bce)
                        add_moving_summary(loss_bce)

                    wd_loss = regularize_cost('.*/W', l2_regularizer(1.0e-5), name='l2_wd_loss')
                    add_moving_summary(wd_loss)

                    cost = tf.add_n(loss_list) + wd_loss
                    self.cost = tf.identity(cost, name='overall_cost')
                    add_moving_summary(self.cost)
                    ####

                    add_param_summary(('.*/W', ['histogram']))   # monitor W

                    #### logging visual sthg
                    orig_imgs = tf.cast(orig_imgs  , tf.uint8)
                    tf.summary.image('input', orig_imgs, max_outputs=1)

                    colored_list = [true] + prob_list + [tf.expand_dims(pen_map, axis=-1)]
                    colored_list = [colorize(feat[...,0], cmap='jet') for feat in colored_list]

                    viz = tf.concat([orig_imgs] + colored_list, 2)
                    tf.summary.image('output', viz, max_outputs=1)

                return
        return Graph

### single_image_infer.py
Contains:
* Inferer class
* Other Methods:
 * process_instance_micro
 * process

Depends on:
* other.py
* Config.py
* postprocessing

In [ ]:
def process(pred, model_mode, ws=True):
    def gen_inst_dst_map(ann):  
        shape = ann.shape[:2] # HW
        nuc_list = list(np.unique(ann))
        nuc_list.remove(0) # 0 is background

        canvas = np.zeros(shape, dtype=np.uint8)
        for nuc_id in nuc_list:
            nuc_map   = np.copy(ann == nuc_id)    
            nuc_dst = distance_transform_edt(nuc_map)
            nuc_dst = 255 * (nuc_dst / np.amax(nuc_dst))       
            canvas += nuc_dst.astype('uint8')
        return canvas
    
    if model_mode != 'dcan':
        assert len(pred.shape) == 2, 'Prediction shape is not HW'
        pred[pred  > 0.5] = 1
        pred[pred <= 0.5] = 0

        # ! refactor these
        ws = False if model_mode == 'unet' or model_mode == 'micronet' else ws
        if ws:
            dist = measurements.label(pred)[0]
            dist = gen_inst_dst_map(dist)
            marker = np.copy(dist)
            marker[marker <= 125] = 0
            marker[marker  > 125] = 1
            marker = binary_fill_holes(marker) 
            marker = binary_erosion(marker, iterations=1)
            marker = measurements.label(marker)[0]

            marker = remove_small_objects(marker, min_size=10)
            pred = watershed(-dist, marker, mask=pred)
            pred = remove_small_objects(pred, min_size=10)
        else:
            pred = binary_fill_holes(pred) 
            pred = measurements.label(pred)[0]
            pred = remove_small_objects(pred, min_size=10)
        
        if model_mode == 'micronet':
            # * dilate with same kernel size used for erosion during training
            kernel = np.array([[0, 1, 0],
                               [1, 1, 1],
                               [0, 1, 0]], np.uint8)
    
            canvas = np.zeros([pred.shape[0], pred.shape[1]])
            for inst_id in range(1, np.max(pred)+1):
                inst_map = np.array(pred == inst_id, dtype=np.uint8)
                inst_map = cv2.dilate(inst_map, kernel, iterations=1)
                inst_map = binary_fill_holes(inst_map)
                canvas[inst_map > 0] = inst_id
            pred = canvas
    else:
        assert (pred.shape[2]) == 2, 'Prediction should have contour and blb'
        blb = pred[...,0]
        blb = np.squeeze(blb)
        cnt = pred[...,1]
        cnt = np.squeeze(cnt)

        pred = blb - cnt # NOTE
        pred[pred  > 0.3] = 1 # Kumar 0.3, UHCW 0.3
        pred[pred <= 0.3] = 0 # CPM2017 0.1
        pred = measurements.label(pred)[0]
        pred = remove_small_objects(pred, min_size=20)
        canvas = np.zeros([pred.shape[0], pred.shape[1]])

        k_disk = np.array([
            [0, 0, 0, 1, 0, 0, 0],
            [0, 0, 1, 1, 1, 0, 0],
            [0, 1, 1, 1, 1, 1, 0],
            [1, 1, 1, 1, 1, 1, 1],
            [0, 1, 1, 1, 1, 1, 0],
            [0, 0, 1, 1, 1, 0, 0],
            [0, 0, 0, 1, 0, 0, 0],
        ], np.uint8)
        for inst_id in range(1, np.max(pred)+1):
            inst_map = np.array(pred == inst_id, dtype=np.uint8)
            inst_map = cv2.dilate(inst_map, k_disk, iterations=1)
            inst_map = binary_fill_holes(inst_map)
            canvas[inst_map > 0] = inst_id
        pred = canvas
        
    return pred

####
def process_instance_micro(pred_map, nr_types=0, output_dtype='uint16'):
    pred_inst = pred_map[...,0].copy()
    pred_inst = process(1-pred_inst, 'micronet')

    if nr_types != 0:
        pred_type = pred_map[..., 1:nr_types]
        pred_type = np.argmax(pred_type, axis=-1)
        pred_type = pred_type+1
        pred_type = np.squeeze(pred_type)

        pred_type_out = np.zeros([pred_type.shape[0], pred_type.shape[1]])               
        #### * Get class of each instance id, stored at index id-1
        pred_id_list = list(np.unique(pred_inst))[1:] # exclude background ID
        pred_inst_type = np.full(len(pred_id_list), 0, dtype=np.int32)
        for idx, inst_id in enumerate(pred_id_list):
            inst_tmp = pred_inst == inst_id
            inst_type = pred_type[pred_inst == inst_id]
            type_list, type_pixels = np.unique(inst_type, return_counts=True)
            type_list = list(zip(type_list, type_pixels))
            type_list = sorted(type_list, key=lambda x: x[1], reverse=True)
            inst_type = type_list[0][0]
            if inst_type == 0: # ! pick the 2nd most dominant if exist
                if len(type_list) > 1:
                    inst_type = type_list[1][0]
            pred_type_out += (inst_tmp * inst_type)
    else:
        pred_type_out = pred_inst.copy()
        pred_type_out[pred_type_out!=0] = 1

    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

def process_instance_hcnet(pred_map, nr_types=0, output_dtype='uint16'):
    pos_map = pred_map[...,-1]*255
    binary_map = pred_map[...,-2]*255
    pred_inst = gaussianmap2binary(pos_map.astype('uint8'), binary_map.astype('uint8'), 150, 40)
    
    if nr_types != 0:
        pred_type = pred_map[..., 1:nr_types]
        pred_type = np.argmax(pred_type, axis=-1)
        pred_type = pred_type + 1
        pred_type = np.squeeze(pred_type)

        pred_type_out = np.zeros([pred_type.shape[0], pred_type.shape[1]])               
        #### * Get class of each instance id, stored at index id-1
        pred_id_list = list(np.unique(pred_inst))[1:] # exclude background ID
        pred_inst_type = np.full(len(pred_id_list), 0, dtype=np.int32)
        for idx, inst_id in enumerate(pred_id_list):
            inst_tmp = pred_inst == inst_id
            inst_type = pred_type[pred_inst == inst_id]
            type_list, type_pixels = np.unique(inst_type, return_counts=True)
            type_list = list(zip(type_list, type_pixels))
            type_list = sorted(type_list, key=lambda x: x[1], reverse=True)
            inst_type = type_list[0][0]
            if inst_type == 0: # ! pick the 2nd most dominant if exist
                if len(type_list) > 1:
                    inst_type = type_list[1][0]
            pred_type_out += (inst_tmp * inst_type)
    else:
        pred_type_out = pred_inst.copy()
        pred_type_out[pred_type_out!=0] = 1

    return pred_inst.astype(output_dtype), pred_type_out.astype(output_dtype)

####
class Inferer(Config):

    def __gen_prediction(self, x, predictor):
        """
        Using 'predictor' to generate the prediction of image 'x'
        Args:
            x : input image to be segmented. It will be split into patches
                to run the prediction upon before being assembled back            
        """    
        step_size = self.infer_avalible_shape
        msk_size = self.infer_avalible_shape
        win_size = self.infer_input_shape
        crop_size = int((self.infer_mask_shape[0] - self.infer_avalible_shape[0])/2)

        def get_last_steps(length, msk_size, step_size):
            nr_step = math.ceil((length - msk_size) / step_size)
            last_step = (nr_step + 1) * step_size
            return int(last_step), int(nr_step + 1)
        
        im_h = x.shape[0] 
        im_w = x.shape[1]

        last_h, nr_step_h = get_last_steps(im_h, msk_size[0], step_size[0])
        last_w, nr_step_w = get_last_steps(im_w, msk_size[1], step_size[1])

        diff_h = win_size[0] - step_size[0]
        padt = diff_h // 2
        padb = last_h + win_size[0] - im_h

        diff_w = win_size[1] - step_size[1]
        padl = diff_w // 2
        padr = last_w + win_size[1] - im_w

        x = np.lib.pad(x, ((padt, padb), (padl, padr), (0, 0)), 'reflect')

        #### TODO: optimize this
        sub_patches = []
        # generating subpatches from orginal
        for row in range(0, last_h, step_size[0]):
            for col in range (0, last_w, step_size[1]):
                win = x[row:row+win_size[0], 
                        col:col+win_size[1]]
                sub_patches.append(win)

        pred_map = deque()
        while len(sub_patches) > self.inf_batch_size:
            mini_batch  = sub_patches[:self.inf_batch_size]
            sub_patches = sub_patches[self.inf_batch_size:]
            mini_output = predictor(mini_batch)[0]
#             print(mini_output.shape)
            mini_output = np.split(mini_output, self.inf_batch_size, axis=0)
            pred_map.extend(mini_output)
        if len(sub_patches) != 0:
            mini_output = predictor(sub_patches)[0]
#             print(mini_output.shape)
            mini_output = np.split(mini_output, len(sub_patches), axis=0)
            pred_map.extend(mini_output)
        
        #### Assemble back into full image
        output_patch_shape = np.squeeze(pred_map[0]).shape
        ch = 1 if len(output_patch_shape) == 2 else output_patch_shape[-1]

        #### Assemble back into full image
        pred_map = np.squeeze(np.array(pred_map))
        
        #### for crop_size !=0\
        if crop_size != 0:
            pred_maps = []
            for i in range(pred_map.shape[0]):
                one_pred_map = pred_map[i][crop_size:-crop_size,crop_size:-crop_size,...]
                pred_maps.append(one_pred_map)
            pred_maps = np.array(pred_maps)
            pred_map = pred_maps
        
        pred_map = np.reshape(pred_map, (nr_step_h, nr_step_w) + pred_map.shape[1:])
        
        pred_map = np.transpose(pred_map, [0, 2, 1, 3, 4]) if ch != 1 else \
                        np.transpose(pred_map, [0, 2, 1, 3])
        pred_map = np.reshape(pred_map, (pred_map.shape[0] * pred_map.shape[1], 
                                         pred_map.shape[2] * pred_map.shape[3], ch))
        pred_map = np.squeeze(pred_map[:im_h,:im_w]) # just crop back to original size
        return pred_map

    ####
    def run(self, file_path):

        model_path = self.inf_model_path

        model_constructor = self.get_model()
        pred_config = PredictConfig(
            model        = model_constructor(),
            session_init = get_model_loader(model_path),
            input_names  = self.eval_inf_input_tensor_names,
            output_names = self.eval_inf_output_tensor_names)
        predictor = OfflinePredictor(pred_config)
        img = cv2.imread(file_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pred_map = self.__gen_prediction(img, predictor)
        return pred_map

### Demo

In [ ]:
tf.test.gpu_device_name()
# Had to use pip install tensorflow-gpu for this to work

In [ ]:
gpus = "0,1,2,3,4,5,6"
n_gpus = len(gpus.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = gpus
# file path for all ROIs that need to be predicted
# dir path for saving predicted results, .mat format files
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

In [ ]:
with open(file_path) as fp:
    lines = fp.read().splitlines() 
img_list = []
for i in lines:
    path = i.split()[0]
    img_list.append(path)
    
inferer = Inferer()

In [ ]:
for img_path in img_list:
    img_name = img_path.split('/')[-1].strip('.png')
    out_file = out_dir + img_path.split('/')[-2]
    out_path = out_file+'/'+img_name+'.mat'
    pred_map = inferer.run(img_path)
    pred_inst, pred_type = process_instance_micro(pred_map, inferer.nr_types)
    labels={'Instance':pred_inst,'Type':pred_type}
    sio.savemat(out_path, labels)
    print(out_path)